## Tweetting like a Trump

### Learning

Questo codice utilizza un dataset dei tweet di Donald Trump per generare nuovi tweet utilizzando gli n-grammi.\
Il dataset originale è stato leggermente pulito da alcuni caratteri speciali con bassa frequenza. tweet_clean.csv\
Vengono ottenuti gli n-grammi dalle frasi; un n-gramma consiste in n parole consecutive, strutturate come $((w_{1}, ..., w_{n-1}), w_{n})$. \
In questo modo, abbiamo già la divisione tra **contesto** e **parola da predire**.\
Vengono aggiunti token speciali come padding per indicare l'inizio `<S>` e la fine `</S>`  di una frase.\
Le probabilità di transizione vengono calcolate per ogni parola dato il suo contesto, contando le occorrenze di ogni n-gramma e di ogni contesto.

##### Probabilità n-gramma: $p(w_i|w_{i-1}...w_{i-(n-1)})= \frac{C(w_{i-(n-1)}...w_{i-1},w_i)}{C(w_{i-1}...w_{i-(n-1)})}$  &nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;Probabilità 2-gramma: $p(w_i|w_{i-1})= \frac{C(w_{i-1},w_i)}{C(w_{i-1})}$


### Decoding
Una frase viene generata scegliendo una parola alla volta dato il contesto precedente, fino alla generazione di `</S>`  o al raggiungimento di una lunghezza massima.\
La scelta può essere **deterministica**: viene scelta la parola con la **probabilità massima**.\
Oppure **semi-randomica**: una parola viene scelta in base alla **distribusione di probabilità**, consentendo la generazione di frasi diverse.

#### Temperatura
La "randomicità" della scelta può essere regolata in base alla **temperatura**, che altera la distribuzione di probabilità con una funzione **softmax**.\
Softmax consente di **intensificare** o **smorzare** la probabilità delle parole, in base alla temperatura [1,1000].\
Per temp=0, la scelta viene forzata in modo deterministico; per temp bassa, la scelta è quasi deterministica; per temp alta, la scelta è randomica.\
Empiricamente: per temp=1000, otteniamo una **distribuzione uniforme**; tuttavia già per temp=100, si ottengono diverse frasi insensate.

#### Contesto Iniziale
Se non viene fornito un contesto iniziale, si parte con n-1 token `<S>` .
Se viene fornito un contesto iniziale, si parte dalle sue ultime n-1 parole; se è troppo corto, vengono aggiunti token `<S>` fino a raggiungere la lunghezza n-1.\
La presenza di un contesto inserito dall'utente crea un problema aggiuntivo: il contesto potrebbe **non essere presente nel dataset**.\
In questo caso, per predire la parola successiva, si cerca di utilizzare un **contesto più breve** utilizzando un (n-1)-gramma.\
Ad esempio, (*some,people,do*) potrebbe non essere presente, ma (*people,do*) potrebbe esserci.\
Se il contesto più piccolo (una singola parola) non è presente, significa che la parola non è presente in tutto il dataset, ovvero è una **parola sconosciuta**\
In questo caso, si cerca di stimare la probabilità di una parola successiva data una parola sconosciuta $P(word|\text{UNK}) = \frac{C(\text{UNK},word)}{C(\text{UNK})}$\
Per questa stima sono state utilizzate le parole che **occorrno una sola volta** nel dataset considerandole "sconosciute"


### Risultati
Il **linguaggio "ripetitivo"** di Donald Trump riesce ad essere colto sufficentemente bene da uno **struemento semplice** come gli n-grammi e con un dataset così ristretto.\
L'utilizzo della **temperatura** per controllare la varianza nella scelta delle parole, permette di ottenere diverse frasi non solo sensate, ma anche plausibili per il personaggio.\
La possibilità di inserire un **contesto inziale** a piacere incrementa ulteriormente la varietà di frasi generabili.\
Alcuni esempi significativi sono stati raccolti in *TrumpTweets_results.txt*\
Empiricamente **i risultati migliori** sono stati ottenuti **con temp [1,50]**. \
Purtroppo per n sempre maggiore non si hanno risultati più soddisfacenti, ma il sistema inzia a ripetere intere frasi del dataset.\
Il codice potrebbe essere computazionalmente ottimizzato per evitare il calcolo di probabilità già precedentemente calcolate o per evitare di cercare contesti smepre più piccoli, ma che già si sa presentano parole sconosciute all'interno.


In [2]:
import pandas as pd
import numpy as np
from collections import Counter
import re
import random

tweets_df=pd.read_csv("../data/tweets_clean.csv",quoting=3)
tweets=tweets_df["text"]

In [3]:
#funzione per tokenizzare una frase in modo opportuno
def tokenize(sentence):
    regex_patterns = [
        r'(?:https?://\S+)',  # Link
        r'(?:@\w+|#\w+)',      # Hashtag o Mention
        r'(?:\.{2,})',        # Insiemi di punti
        r"(?:[A-Za-z0-9&]+(?:[-’']+[A-Za-z0-9&]+)*)",  # Parole contratte
        r'(?:[.!?;:“”\-])'  # Punteggiature
    ]
    combined_pattern = '|'.join(regex_patterns)
    tokens = re.findall(combined_pattern, sentence)
    return tokens

#Esempio di utilizzo
test_sentence = "“ciao” -disse @pippo \"come va?.....\" rispose l'altro; vai su : https://www.example.com? #Buonaidea! "
print(tokenize(test_sentence))

['“', 'ciao', '”', '-', 'disse', '@pippo', 'come', 'va', '?', '.....', 'rispose', "l'altro", ';', 'vai', 'su', ':', 'https://www.example.com?', '#Buonaidea', '!']


In [4]:
#data una frase ritorna una lista di tuple di n grammi nella forma [(context, word), ...]  in cui context è formato da n-1 token e word è il token successivo
#vengono aggiunti n-1 token <S> e </S> all'inizio e alla fine della frase
#esempio per n=3 "ciao a tutti"  [(('<S>', '<S>'), 'ciao'), (('<S>', 'ciao'), 'a'), (('ciao', 'a'), 'tutti'), (('a', 'tutti'), '</S>'),(('tutti', '</S>'), '</S>')]
def get_ngrams(sentence,n): 
    if n<2: n=2 #forzo n>=2  (non ha senso avere meno di una parola come contesto)
    tokens = (n-1)*['<S>']+tokenize(sentence)+(n-1)*['</S>']
    ngrams = []
    for i in range(n - 1, len(tokens)):
        context = tuple(tokens[i-(n-1)+j] for j in range(n - 1))   #dato l'indice i, torno indietro di n-1 token e prendo i token successivi
        ngrams.append((context, tokens[i]))
    return ngrams

#esempio di utilizzo
print(get_ngrams("ciao a tutti",1))

[(('<S>',), 'ciao'), (('ciao',), 'a'), (('a',), 'tutti'), (('tutti',), '</S>')]


In [ ]:
#calcola le occorenze per ogni ngramma ((context),word) e per ogni contesto (context)
#se n=1 considera solo le parole con occorenza 1 (sconosciute) e le raggruppa sotto il contesto ('UNK',). ignora le parole con occorenza >1
def counts_occurence(tweets,n=3):
    ngrams_counts={} #occorenze degli ngrammi ((context),word) 
    context_counts={}#occorenze dei contesti (context)

    if n==1: #se n=1 ottengo la lista di parole sconosciute
        words= ' '.join(tweets).lower().split()  
        words_count = Counter(words)  
        unk_words = [word for word, count in words_count.items() if count == 1]

    #per ogni tweet ottengo gli n-grammi e aggiorno i contatori
    for tweet in tweets:
        ngrams=get_ngrams(tweet,n)
        for context, word in ngrams:
            if n == 1:#se n=1 considero le parole sconosciute
                if context[0] in unk_words:
                    context = ('UNK',)
                else:
                    continue  #salto le parole conosciute
            #incremento i contatori    
            ngrams_counts[(context, word)] = ngrams_counts.get((context, word), 0) + 1
            context_counts[context] = context_counts.get(context, 0) + 1
    return context_counts, ngrams_counts

#Esempio di utilizzo
context_counts, ngrams_counts = counts_occurence(tweets,2)
print(ngrams_counts)
print(context_counts)

##### Probabilità 2-gram=$p(w_i|w_{i-1})= \frac{C(w_{i-1},w_i)}{C(w_{i-1})}$

##### Probabilità n-gram $p(w_i|w_{i-1}...w_{i-(n-1)})= \frac{C(w_{i-(n-1)}...w_{i-1},w_i)}{C(w_{i-1}...w_{i-(n-1)})}$

In [ ]:
#date le occorenze di un contesto e di un ngramma calcola la probabilità P(word|context) = count(context,word) / count(context)
#se le occorenze riguardavano le parole sconosciure calcola la probabilita P(word|UNK) = count(UNK,word) / count(UNK)
#ritorna un dizionario di probabilità {ngramma: probabilità}
def calculate_probabilities(context_counts, ngrams_counts):
    ngrams_prob= {}# P(ngramma) = count(ngramma) / count(context)
    for ngram, ngram_count in ngrams_counts.items():
        context = ngram[0]
        ngrams_prob[ngram] = ngram_count / context_counts[context]
    return ngrams_prob
        
ngrams_prob = calculate_probabilities(context_counts, ngrams_counts)
print(ngrams_prob)

In [48]:
#data una lista di probabilità, resituisce una lista alterata in base alla temperatura [1-1000]
#se la temperatura è alta, le probabilità vengono appiattite (distribuzione uniforme), se è bassa le probabilità vengono esaltate
def softmax(x, temperature):
    temperature=temperature/1000
    e_x = np.exp((x - np.max(x)) / temperature)
    return e_x / e_x.sum()

#----------------------------------------------------------
#data una porzione di testo e un dizionario di probabilità, genera la parola successiva usando un n gramma
#la scelta della parola può essere deterministica (temp=0) o semi-randomica in base alla temperatura, (più è alta più la scelta è randomica)
#se il contesto per ngramma non è presente nel dizionario di probabilità, viene provato a generare una parola con un contesto più corto (n-1)
#quando n=1 si considera il contesto ('UNK',) e viene generata una parola in base alla distribuzione delle parole che seguono le parole sconosciute(con occorenza 1)
def generate_word(n,text, ngrams_prob=None, temp=0):
    #se non è presente il dizionario di probabilità viene calcolato
    if ngrams_prob is None:
        context_counts, ngrams_counts = counts_occurence(tweets,n)
        ngrams_prob = calculate_probabilities(context_counts, ngrams_counts)
    
    if n==1: context = ('UNK',) #se n=1, la parola è sconosciuta
    else: context = tuple(text[-(n-1):]) #prendo gli ultimi n-1 token del testo

    #se il contesto non è presente nel testo provo a generare una parola con un contesto più corto
    if [ngram for ngram in ngrams_prob if ngram[0] == context]==[] :
            return generate_word(n-1,text, ngrams_prob=None, temp=temp) 
    else:#genero la parola
        if temp==0:#scelta deterministica, prendo l'ngramma con probabilità massima dato il contesto
            best_ngram = max((ngram for ngram in ngrams_prob if ngram[0] == context), key=lambda ngram: ngrams_prob[ngram])
        else:#scelta semi-randomica, scelgo l'ngramma usando le probabilità come pesi alterati dalla temperatura
            ngrams=[ngram for ngram in ngrams_prob if ngram[0] == context]
            probs=[ngrams_prob[ngram] for ngram in ngrams]
            probs=softmax(probs,temp)
            best_ngram = random.choices(ngrams, weights=probs, k=1)[0]
    return best_ngram[1]

#----------------------------------------------------------
#data una porzione di testo, genera un testo di lunghezza massima max_len o fino a che non viene generato </S>
def generate_text(context="",n=3, max_len=10,temp=0):
    context_counts, ngrams_counts = counts_occurence(tweets,n)
    ngrams_prob = calculate_probabilities(context_counts, ngrams_counts)

    #inzializzazione del contesto iniziale (aggiunta di token <S>)
    context=tokenize(context)
    if len(context)<n-1:
        text = (n-1-len(context))*['<S>']+context
    else:
        text = context

    #generazione del testo
    for i in range(max_len):
        next_word = generate_word(n,text, ngrams_prob, temp)
        text.append(next_word)
        if next_word == '</S>': #se viene generato il token finale interrompo
            break     

    text_filtered = [item for item in text if (item != '</S>' and item != '<S>')]
    return ' '.join(text_filtered)

#esempio di utilizzo
for i in range(2,10):
    print("n = "+str(i),generate_text(context="",n=i,max_len=30,temp=10))

n = 2 I have no idea how a loser !
n = 3 The Fake News Losers ! https://t.co/3RHcBZogms
n = 4 I have many great people but also an amazing number of haters and losers !
n = 5 @Ashley33331 True but the losers don't know that - thanks .
n = 6 I have many great people but also an amazing number of haters and losers responding to my tweets-why do these lowlifes follow-nothing to do !
n = 7 I have watched sloppy Graydon Carter fail and close Spy Magazine and now am watching him fail at @VanityFair Magazine . He is a total loser !
n = 8 I have nothing to do with Atlantic City-sold years ago great timing . For losers and haters I NEVER went bankrupt . Plus 10 billion sorry
n = 9 I have nothing to do with Atlantic City-sold years ago great timing . For losers and haters I NEVER went bankrupt . Plus 10 billion sorry
